# FCFG_generator
Sentence generator for Context Free Grammars with features.
This is based on the nltk Feature Context Free Grammars. However, that system doesn't generate sentences correctly. It seems that the feature constraints only work for parsing, not generating. Possibly this is just a bug, but it seems like the nltk infrastructure may not support the kind of unification that is required. So I've writting that from scratch. So I only use the grammar parser and FCFC data structure from nltk. But that should mean that this generator will work with any FCFG created with nltk.


## Code for FCFGs and Feature Unification
You don't need to understand the implmentation to create FCRG grammars and use the FCFG sentence generator.

In [2]:
import nltk, copy, random, copy

# Go through all feature values of all nodes in a list of nodes.
# Return a mapping from distinct variable names to 'slots'.
# The slots are empty lists into which values can later be inserted.
def get_nodelist_variable_map(nodelist):
    varmap = {}
    for node in nodelist:
      if type(node)==str: continue
      for feature in node:
        if type(node[feature]) == nltk.sem.logic.Variable:
          varmap[node[feature].name] = []
    return varmap

## Copy nodelist and replace all variables with slots
## Instances of the same variable will get the same slot value.
def nodelist_replace_variables(nodelist):
    nodelist = copy.deepcopy(nodelist)
    varmap = get_nodelist_variable_map(nodelist)
    for node in nodelist:
      if type(node)==str: continue
      for feature in node:
          if type(node[feature]) == nltk.sem.logic.Variable:
              node[feature] = varmap[node[feature].name]
    return nodelist

## Copy a node, replacing Variable feature values with 'slots'
def replace_node_vars(node):
  if type(node)==str: return node  # quick return for terminals
  ## Otherwise just do it as a nodelist with one element
  return nodelist_replace_variables([node])[0]

## Copy a production rule, replacing Variable feature values with 'slots'
def replace_production_vars(prod):
  ## Turn the rule into a node list
  nodes = [prod.lhs()] + list(prod.rhs())
  nodes = nodelist_replace_variables(nodes)
  ## Turn the result back into a production rule
  return nltk.Production(nodes[0], nodes[1:])

## Unpack nested slots untill you get either empty list (variable slot)
## Or you get a value (the slot variable has been instantiated)
## If v is not a slot you just get back v.
def unslot(v):
  while type(v)== list:
    if v == []: return v # return the slot
    v = v[0]             # unpack list
  return v               # return any non-list value

def unify_values(v1, v2):
      v1 = unslot(v1)
      v2 = unslot(v2)
      # Case of both str values
      if type(v1)==str and type(v2)==str:
          return v1==v2
      # Cases of one str and one slot (variable) feature value
      # Insert the str into the slot.
      if type(v1)==str:
            v2.insert(0,v1)
      elif type(v2)==str:
            v1.insert(0,v2)
      else:
          # Remaining case: both are slots
          slot = []   # Bind by adding new slot within each of them:
          v1.insert(0, slot)
          v2.insert(0, slot)
      return True

def unify_nodes( n1, n2 ):
    for feature in n1:
        if feature in n2:
            if not unify_values( n1[feature], n2[feature] ):
                return False
    return True

def node_type(node):
  return list(node.items())[0][1] # is there an easier way??


## The main generator function
def generate_random_from_grammar(grammar, node=None, depth=0, debug=False):
    def dbprint(*args):
      if debug: print(*args)

    if node == None:
      node = grammar.start()
      node = replace_node_vars(node)
    dbprint("***DEPTH:", depth)
    dbprint("* Generating from node:\n", node)
    dbprint("--- end of node ---") 
    rule_options = grammar.productions(lhs=node)
    if debug:
      print("Rule options:")
      for r in rule_options:
          print(r)
    random.shuffle(rule_options) # First shuffle then pick first match (for speed)
    chosen_rule = None
    for rule in rule_options:
      rule = replace_production_vars(rule) # gives a copy with var slots added
      rule_head = rule.lhs()
      # we test that a copy of the node will unify
      # (Don't want to start binding the acual node in case it fails half way/)
      node_copy = copy.deepcopy(node) 
      if unify_nodes( node_copy, rule_head ):  # It's a Match!
        chosen_rule = rule
        break
    if not chosen_rule:
      print("!!! No applicable rules were found to expand node:\n", node)
      return False
    dbprint("chosen_rule:", chosen_rule)
    ## NOW UNIFY THE CHOSEN RULE HEAD WITH THE NODE
    unify_nodes( node, chosen_rule.lhs() )
    ## Vars in the chosen rule will now be instantiated as required
    ## (but this is just a copy of the actual rule in the grammar)
    ## Now expand the children in the matched and instantiated rule copy:
    #children = [node_type(node)]
    children = []
    for item in chosen_rule.rhs():
      dbprint("Now instantiate item, type:", item, type(item))
      if type(item) == nltk.grammar.FeatStructNonterminal:
        child = generate_random_from_grammar(grammar, node=item, depth=depth+1)
        children.append(child)
      else:
        children.append(item)
    ## Return structure as list of node type and children
    return [node_type(node)] + children

# Now using unpack istead of unpack because I've added node types
# to the structure and need to throw them away.    
def flatten(L):
    if L == []: return L
    if type(L[0]) == list:
        return flatten(L[0]) + flatten(L[1:])
    return [L[0]] + flatten(L[1:])

def unpack(struct):
  if struct == [] : return []
  if type(struct) == list:
    result = []
    for x in struct[1:]:
      result.extend(unpack(x))
    return result
  return [struct]

In [3]:

def fcfg_from_string( string ):
    return nltk.grammar.FeatureGrammar.fromstring(string)

def make_node( ntype, **kwargs ):
    return nltk.grammar.FeatStructNonterminal( ntype, **kwargs )

def show_random_sentence(grammar, 
                         node=None, # start node
                         n=1,       # number of sentences
                         show_sentence=True, show_structure=True, debug=False):
    if type(grammar) == str:
      grammar = fcfg_from_string( grammar )
    for _ in range(n):
      structure = generate_random_from_grammar(grammar, node=node, debug=debug)
      if show_structure: 
        print(structure)
      if show_sentence:
        #words = flatten(structure)
        words = unpack(structure)
        words = [(word if word else "!NO_EXPANSION!") for word in words]
        words[0] = words[0].title()
        sentence = " ".join(words) + "."
        print(sentence)

## Example Grammars
The `EG_GRAMMAR` string below illustrates FCFG grammar specification. You can devise whatever grammar you fancy within the general framework of Context Free Grammar rules augmented with _features_ and _feature variables_.

In [4]:
EG_GRAMMAR = """
% start S
S[ATYPE=?at] -> NP[NUM=?n,ATYPE=?at] VP[NUM=?n,ATYPE=?at]

NP[NUM=sg,ATYPE=?at] -> PN[NUM=sg,ATYPE=?at]
NP[NUM=?n,ATYPE=?at] -> Det[NUM=?n] CN[NUM=?n,ATYPE=?at]

PN[NUM=sg,ATYPE=intelligent] -> 'Bob' | 'John' | 'Alice' | 'Mary' | 'Karen'

Det -> 'the' | 'some'
Det[NUM=sg] -> 'a' | 'one'
Det[NUM=pl] -> 'several' | 'two'

CN[NUM=sg,ATYPE='animate'] -> 'dog'  | 'cat' | 'horse'
CN[NUM=pl,ATYPE='animate'] -> 'dogs' | 'cats' |'horses'
CN[NUM=sg,ATYPE='intelligent'] -> 'student' 
CN[NUM=pl,ATYPE='intelligent'] -> 'students'

VP[NUM=sg,ATYPE='intelligent'] -> 'sings' | 'laughs'
VP[NUM=pl,ATYPE='intelligent'] -> 'sing'  | 'laugh'
VP[NUM=sg,ATYPE='animate'] -> 'runs' | 'jumps'
VP[NUM=pl,ATYPE='animate'] -> 'run'  | 'jump'
"""


Now we can generate some random sentences based on the grammar.

In [ ]:
show_random_sentence(EG_GRAMMAR, n=10, node=None, show_structure=False, debug=False)

Alice laughs.
Mary laughs.
John laughs.
Mary sings.
The student laughs.
Several cats run.
Some dogs run.
Karen sings.
Two horses jump.
Two dogs jump.


You can set `show_structure=True` to see the phrase structure of the sentences.

In [5]:
show_random_sentence(EG_GRAMMAR, n=5, node=None, show_structure=True)

['S', ['NP', ['Det', 'several'], ['CN', 'horses']], ['VP', 'run']]
Several horses run.
['S', ['NP', ['Det', 'several'], ['CN', 'students']], ['VP', 'laugh']]
Several students laugh.
['S', ['NP', ['PN', 'Alice']], ['VP', 'sings']]
Alice sings.
['S', ['NP', ['PN', 'John']], ['VP', 'sings']]
John sings.
['S', ['NP', ['Det', 'some'], ['CN', 'cat']], ['VP', 'runs']]
Some cat runs.


You can make a start node with some features set (as if keyword args). The Default is to start at S with no features set. So if we want to generate sentences involving intelligent actions we could create the following:

In [6]:
s_intelligent = make_node('S', ATYPE='intelligent')

Then you can generate sentences (or other expressions) starting with that type of node:

In [7]:
show_random_sentence(EG_GRAMMAR, n=5, node=s_intelligent, show_structure=False, debug=False)

Several students sing.
One student laughs.
Karen laughs.
One student laughs.
Bob laughs.


## Customised Coarse-Grained Grammars
You don't necessarily need to use the GFSGs framework to specify the detailed syntactic structure of natual language. They are also useful if we want to specify classes of example sentences over a limited range of patterns that we may be interested in. For example, we could generate examples to be used for testing the performance of Language Models.

In [8]:
POURING_GRAMMAR = """
S -> LEFT CON RIGHT
CON -> 'because'
LEFT -> 'The' PERSON_ADJ PERSON 'poured the' WINE_ADJ 'wine into the cup'
PERSON_ADJ -> 'happy' | 'sad' | 'unlucky'
PERSON -> 'cook' | 'student'
RIGHT -> 'they were' REASON_ADJ
WINE_ADJ -> 'red' | 'white'
REASON_ADJ -> 'thirsty' | 'in need of some relaxation'
"""

show_random_sentence(POURING_GRAMMAR, n=20, node=None, show_structure=False, debug=False)

The unlucky cook poured the white wine into the cup because they were thirsty.
The unlucky student poured the red wine into the cup because they were in need of some relaxation.
The happy student poured the white wine into the cup because they were thirsty.
The sad cook poured the red wine into the cup because they were thirsty.
The happy student poured the red wine into the cup because they were thirsty.
The unlucky student poured the white wine into the cup because they were in need of some relaxation.
The happy cook poured the red wine into the cup because they were thirsty.
The happy cook poured the red wine into the cup because they were in need of some relaxation.
The happy student poured the red wine into the cup because they were thirsty.
The happy student poured the red wine into the cup because they were thirsty.
The happy student poured the white wine into the cup because they were in need of some relaxation.
The sad student poured the red wine into the cup because they were

In [ ]:
# test